In [2]:
import copy
import os
import numpy as np
import pandas as pd
import zipfile
from simple_downloader import download
from tqdm.notebook import tqdm
from pathlib import Path
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, Subset, DataLoader

In [3]:
class UCMerced(Dataset):
    def __init__(self, root_dir, img_transform=None, multilabel=True):

        self.root_dir = root_dir
        self.images_path = os.path.join(root_dir, "Images")
        self.class_names = sorted(
            [cl for cl in os.listdir(self.images_path) if not cl.startswith(".")]
        )
        self.img_paths, self.img_labels = self.init_dataset()
        self.img_transform = img_transform

        if multilabel:
            self.img_labels = self.read_multilabels()  # important for loss calculation
            self.img_labels = self.img_labels.astype(float)

    def init_dataset(self):
        img_paths, img_labels = [], []
        for cl_id, cl_name in enumerate(self.class_names):
            cl_path = os.path.join(self.images_path, cl_name)

            for img in sorted(os.listdir(cl_path)):
                img_path = os.path.join(cl_path, img)
                img_paths.append(img_path)
                img_labels.append(cl_id)

        return img_paths, img_labels

    def read_multilabels(self):
        label = pd.read_excel("./Hw3_data/UCMerced_LandUse/multilabels/LandUse_Multilabeled.xlsx")
        label = label.set_index("IMAGE\LABEL")
        label = np.array(label)
        return label

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.img_labels[idx]

        img = Image.open(img_path).convert("RGB")
        if self.img_transform is not None:
            img = self.img_transform(img)

        return dict(img=img, label=label)

    def __len__(self):
        return len(self.img_paths)

In [4]:
class MetricTracker(object):
    """Computes and stores the average and current value."""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
def get_device(cuda_int):
    """Get Cuda-Device. If cuda_int < 0 compute on CPU."""
    if cuda_int < 0:
        print("Computation on CPU")
        device = torch.device("cpu")
    elif torch.cuda.is_available():
        print("Computation on CUDA GPU device {}".format(cuda_int))
        device = torch.device("cuda:{}".format(cuda_int))
    return device

In [6]:
from torchvision import datasets ,models , transforms
import json
from torch.utils.data import Dataset, DataLoader ,random_split
from PIL import Image
from pathlib import Path
from torchvision.models import ResNet18_Weights
from torch.optim import Adam

def get_dataset(root_dir, tr_transform,seed=1, multilabel=True):
    valid_no = int(2100*0.1)
    train_no = int(2100*0.7) 
    test_no = int(2100*0.2)
    
    """
    Parameter
    ---------
    root_dir     : path to UCMerced Dataset
    tr_transform : transformation for training data
    te_transform : transformation for training data
    set_sizes    : list of percentage of either train-test or train-val-test (sum to 100)

    Output
    ------
    sets for train and test, optionally also val if len(set_sizes)==3
    """
    ucm_dataset_tr = UCMerced(root_dir, img_transform=tr_transform, multilabel=multilabel)
    #ucm_dataset_te = UCMerced(root_dir, img_transform=te_transform, multilabel=multilabel)
    
    trainset ,validset, testset  = random_split(ucm_dataset_tr, [train_no, valid_no, test_no])
    
    return trainset ,validset, testset
    
    
    #idx_list = split_ucm_indices(set_sizes, seed=seed)

   # train_set = Subset(ucm_dataset_tr, idx_list[0])
    #test_set = Subset(ucm_dataset_te, idx_list[-1])

    #if len(idx_list) > 2:
     #   val_set = Subset(ucm_dataset_te, idx_list[1])
      #  return train_set, val_set, test_set
    #else:
     #   return train_set, test_set

In [7]:
cuda_device = get_device(-1)

Computation on CPU


In [8]:
batch_size = 64
learning_rate = 0.001
epochs = 20
num_cls = 21

In [9]:
ucm_mean = [0.595425, 0.3518577, 0.3225522]
ucm_std = [0.19303136, 0.12492529, 0.10577361]

#So we resize the images so all image have same size
#This is how we transform all images

#We do it for both training and testing data
#For further understanding, pls refer to the following webste: https://www.programcreek.com/python/example/104832/torchvision.transforms.Compose

tr_transform = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=ucm_mean, std=ucm_std),
    ]
)

te_transform = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=ucm_mean, std=ucm_std),
    ]
)

In [10]:
trainset, valset, testset = get_dataset(
    "./Hw3_data/UCMerced_LandUse",
    tr_transform=tr_transform,
    multilabel=True
)

In [11]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [12]:
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(512, 17) #21 - number of classes
model.to(cuda_device)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/jovyan/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
import torch.optim as optim
from torch.optim import lr_scheduler


#Criterion
criterion = nn.BCEWithLogitsLoss().to(cuda_device)
#criterion = nn.CrossEntropyLoss()




# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [14]:
def train(model, train_loader, val_loader, optimizer, criterion, epochs, device, early_stop=False):
    train_losses, val_losses = [], []
    accuracy_scores = []
    best_model = copy.deepcopy(model)
    best_acc = 0
    best_epoch = 1

    for epoch in range(1, epochs + 1):

        print("Epoch {}/{}".format(epoch, epochs))
        print("-" * 10)

        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, report, _ = val_epoch(model, val_loader, criterion, device)
        overall_acc = report["accuracy"]

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        accuracy_scores.append(overall_acc)

        if best_acc < overall_acc:
            best_acc = overall_acc
            best_epoch = epoch
            best_model = copy.deepcopy(model)

        if epoch - best_epoch > 10 and early_stop:
            break

    return best_model, train_losses, val_losses, accuracy_scores

In [15]:
def train_epoch(model, train_loader, optimizer, criterion, device):
    loss_tracker = MetricTracker()
    acc_tracker = MetricTracker()
    model.train()

    tqdm_bar = tqdm(train_loader, desc="Training: ")
    for batch in tqdm_bar:

        images = batch["img"].to(device)
        labels = batch["label"].to(device)
        batch_size = images.size(0)
        optimizer.zero_grad()

        logits = model(images)
        loss = criterion(logits, labels)
        #loss = criterion(logits, labels.float())
        loss.backward()
        optimizer.step()

        probs = F.softmax(logits, dim=1)
        loss_tracker.update(loss.item(), batch_size)

        _, predicted = torch.max(probs.data, 1)
        #batch_acc = (predicted == labels).sum().item() / batch_size
        #acc_tracker.update(batch_acc, batch_size)
        tqdm_bar.set_postfix(loss=loss_tracker.avg, accuracy=acc_tracker.avg)
        tqdm_bar.set_postfix(loss=loss_tracker.avg)

    return loss_tracker.avg

In [123]:
def val_epoch(model, val_loader, criterion, device):
    loss_tracker = MetricTracker()
    acc_tracker = MetricTracker()
    model.eval()

    y_pred = []
    y_true = []
    

    with torch.no_grad():
        tqdm_bar = tqdm(val_loader, desc="Validation: ")
        for batch in tqdm_bar:

            images = batch["img"].to(device)
            labels = batch["label"].to(device)
            batch_size = images.size(0)

            logits = model(images)
            probs = F.softmax(logits, dim=1)
            loss = criterion(logits, labels)
            loss_tracker.update(loss.item(), batch_size)

            _, predicted = torch.max(probs.data, 1)
           # batch_acc = (predicted == labels).sum().item() / batch_size
            #acc_tracker.update(batch_acc, batch_size)
            acc_tracker.update(batch_size)

            y_pred += predicted.tolist()
            y_true += labels.tolist()
            tqdm_bar.set_postfix(loss=loss_tracker.avg, accuracy=acc_tracker.avg)
            

    report = classification_report(y_true, y_pred, zero_division=0, output_dict=True)
    conf_mat = confusion_matrix(y_true, y_pred, normalize="true")
        
    return loss_tracker.avg, report, conf_mat

In [124]:
eval_accuracies = []

In [127]:
best_model, train_losses, val_losses, accuracy_scores = train(
    model,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    epochs=3,
    device=cuda_device,
)
eval_accuracies.append(accuracy_scores)

Epoch 1/3
----------


Training:   0%|          | 0/23 [00:00<?, ?it/s]

Validation:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [128]:
#Testing val_epoch to test results
#Check the following link: https://discuss.pytorch.org/t/how-to-extract-probabilities/2720/9

def val_epoch(model, val_loader, criterion, device):
    loss_tracker = MetricTracker()
    acc_tracker = MetricTracker()
    model.eval()

    y_pred = []
    y_true = []
    

    with torch.no_grad():
        tqdm_bar = tqdm(val_loader, desc="Validation: ")
        for batch in tqdm_bar:

            images = batch["img"].to(device) #Array of images
            labels = batch["label"].to(device) #Array of labels
            batch_size = images.size(0)

            logits = model(images)
            probs = F.softmax(logits, dim=1)
            loss = criterion(logits, labels)
            
            loss_tracker.update(loss.item(), batch_size)
            predicted_prob, predicted_labels = torch.topk(probs.data, 17)
            #predicted_prob, predicted_labels = torch.max(probs.data, 1)
            
            
           # batch_acc = (predicted == labels).sum().item() / batch_size
            #acc_tracker.update(batch_acc, batch_size)
            acc_tracker.update(batch_size)

            y_pred += predicted_labels.tolist()
            y_true += labels.tolist()
            
            y_pred_arr = np.array(y_pred)
            y_true_arr = np.array(y_true)
            
        
        return images, labels, y_pred_arr, y_true_arr, logits, probs, predicted_labels
          
            

In [131]:
#Idea taken from the following website: https://colab.research.google.com/github/kmkarakaya/ML_tutorials/blob/master/Multi_Label_Model_Evaulation.ipynb#scrollTo=H8WBPN1OxXmw
# We have 210 values because our validation takes 10% of our data so we get 210 values out of 2100
im, lb, y_prediction, y_true, log, probs, pre = val_epoch(model, val_loader, criterion, cuda_device)
print("Y true values are: ", y_true)
print(y_true.shape)

print("Y prediction values are: ", y_prediction)
print(y_prediction.shape)




Validation:   0%|          | 0/4 [00:00<?, ?it/s]

Y true values are:  [[0. 0. 0. ... 0. 1. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
(210, 17)
Y prediction values are:  [[10  9  3 ... 11  4 14]
 [ 8 10  1 ...  9 16 12]
 [10  3  8 ...  0  4  9]
 ...
 [10 11  3 ...  6 13  5]
 [ 3 10 15 ... 13  0  4]
 [10  8  1 ...  7 13  6]]
(210, 17)


## Test Alpha

In [132]:
print("Images", im)
print("Images shape", im.shape)

print("Label", lb)
print("Label shape", lb.shape)

Images tensor([[[[-0.3826, -0.3217, -0.3420,  ..., -0.8905, -1.2562, -1.3984],
          [-0.2201, -0.2201, -0.1795,  ..., -2.1298, -2.3939, -2.4548],
          [-0.1388, -0.1591, -0.0779,  ..., -2.1298, -1.6828, -1.1952],
          ...,
          [-0.5248, -0.4639, -0.5451,  ..., -0.0372,  0.0643, -0.0372],
          [-0.5655, -0.5858, -0.6264,  ...,  0.0440, -0.1795, -0.1795],
          [-0.7077, -0.6467, -0.6061,  ..., -0.2607,  0.0237,  0.2472]],

         [[ 1.1388,  1.1702,  1.0446,  ...,  0.9504,  0.4482,  0.1342],
          [ 1.3271,  1.3271,  1.3271,  ..., -1.0900, -1.5609, -1.7492],
          [ 1.3585,  1.3585,  1.4213,  ..., -1.6865, -1.2470, -0.7447],
          ...,
          [ 1.3271,  1.4213,  1.4527,  ...,  2.2061,  2.4258,  2.2375],
          [ 1.4527,  1.4213,  1.3899,  ...,  2.3002,  1.9549,  1.9549],
          [ 1.3585,  1.3271,  1.3585,  ...,  1.8294,  2.2061,  2.5200]],

         [[ 1.6220,  1.6591,  1.5849,  ...,  1.5849,  1.0288,  0.7322],
          [ 1.8445,  1.

In [110]:
print("Logits:", log)
print("Logits shape:", log.shape)

print("Probs:", probs)
print("Probs Shape:", probs.shape)

Logits: tensor([[ -5.1581,   0.3914,   1.7470,  -1.5012,  -6.2783,  -5.2867,  -7.0403,
          -6.6120,  -1.2838,  -6.2936,   3.0286,  -2.0860,  -5.5985,  -6.6849,
           3.9404,  -1.2268,  -8.9678],
        [ -7.0603,  -2.4734,   1.5256,  -2.7421,  -6.1616,  -1.2679,  -5.9548,
          -6.2588,   4.5558,  -6.1958,   2.0909,  -3.4977,  -6.5851,  -5.2587,
          -5.4385,   3.2597,  -4.7059],
        [  3.8615,  -3.4650,   1.2770,  -0.7040,  -5.6518,  -7.1856,  -6.1340,
          -7.5348,  -1.6172,  -5.9145,   5.6845,  -3.7778,  -6.2146,  -5.4458,
          -1.9188,  -5.9091,  -7.4834],
        [ -2.6568,  -2.6369,  -4.5690,  -2.2128,  -3.7092,  -6.8848,  -4.9252,
          -5.0502,  -1.2266,  -6.6037,  -0.8466,   1.4938,   1.6582,  -3.8813,
          -2.9261,  -3.7825,  -3.1287],
        [ -6.0245,  -1.0564,   2.3203,   1.2047,  -6.4674,  -2.1483,  -7.1153,
          -6.0587,   2.4351,  -5.8091,   2.9217,  -7.1310,  -6.7172,  -6.4234,
          -2.8768,   0.7744,  -8.9534],
  

In [49]:
print("Prediction:", pre)
print("Prediction shape:", pre.shape)

Prediction: tensor([14,  8, 10, 12, 10,  8, 15, 10, 10,  2, 10,  8,  2, 10, 10, 10, 15, 10])
Prediction shape: torch.Size([18])


# Test Bravo

In [ ]:
# We can normalize the y_prediction values between 0 & 1
y_prediction_normalize = []
min_val = min(y_prediction)
max_val = max(y_prediction)
for i in range(len(y_prediction)):
    z = ((y_prediction[i] - min_val)/ (max_val -min_val))
    y_prediction_normalize.append(z)
    
y_prediction_normalize
    
    

In [ ]:
# Threshold: Let's assume we are using 0.5 as the threshold for prediction

y_pred_final=[]
for sample in  y_prediction_normalize:
    if sample > 0.5:
        y_pred_final.append(1)
    else:
        y_pred_final.append(0)
        
y_pred_final = np.array(y_pred_final)
y_pred_final

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_true, y_pred_final)

In [ ]:
def prettify_confusion_matrix(conf_mat, class_names):
    plt.subplots(1, 1, figsize=(11, 7))
    sns.heatmap(
        conf_mat,
        cmap="viridis",
        fmt="g",
        xticklabels=class_names,
        yticklabels=class_names,
        annot=True,
    )

In [ ]:
import numpy as np
y_t = np.array([[0, 1, 1, 1],[0,0,1,0],[1,1,0,0]])
y_p = np.array([[0, 1, 0, 1],
       [0, 1, 1, 1],
       [1, 0, 1, 1]])

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
st = multilabel_confusion_matrix(y_t, y_p)
print(st)

In [ ]:
abel_names = ['label A', 'label B', 'label C', 'label D']

print(classification_report(y_t, y_pr,target_names=label_names))